# Fashion MNIST via TensorFlow #

## Preliminaries ##

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

import numpy as np
import matplotlib.pyplot as plt
import _pickle
import gzip

with gzip.open('./data/fashion-mnist.pkl.gz', 'rb') as fp:
    (train_images, train_labels, test_images, test_labels) = \
    _pickle.load(fp)

In [2]:
train_images_reshape = train_images.reshape(train_images.shape[0],
                                            28, 28, 1)
test_images_reshape = test_images.reshape(test_images.shape[0],
                                         28, 28, 1)

## Simplistic Network ##

In [3]:
model = Sequential([
    Flatten(input_shape=(28,28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [5]:
model.fit(train_images, train_labels,
          validation_data=(test_images, test_labels),
          epochs=10)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 17s 275us/sample - loss: 0.5025 - acc: 0.8247 - val_loss: 0.4094 - val_acc: 0.8529
Epoch 2/10
60000/60000 [==============================] - 12s 202us/sample - loss: 0.3736 - acc: 0.8647 - val_loss: 0.3917 - val_acc: 0.8598
Epoch 3/10
60000/60000 [==============================] - 5s 79us/sample - loss: 0.3354 - acc: 0.8781 - val_loss: 0.3638 - val_acc: 0.8705
Epoch 4/10
60000/60000 [==============================] - 5s 85us/sample - loss: 0.3112 - acc: 0.8859 - val_loss: 0.3531 - val_acc: 0.8733
Epoch 5/10
60000/60000 [==============================] - 6s 98us/sample - loss: 0.2941 - acc: 0.8905 - val_loss: 0.3481 - val_acc: 0.8743
Epoch 6/10
60000/60000 [==============================] - 5s 77us/sample - loss: 0.2814 - acc: 0.8941 - val_loss: 0.3421 - val_acc: 0.8771
Epoch 7/10
60000/60000 [==============================] - 7s 111us/sample - loss: 0.2676 - acc: 0.9008 - val_los

In [7]:
keras.backend.image_data_format()

'channels_last'

## Model 2 ##

In [8]:
model2 = Sequential([
    Conv2D(32, [5, 5], input_shape=(28, 28, 1), activation='relu'),
    BatchNormalization(axis=-1),
    Conv2D(32, [5, 5], activation='relu'),
    BatchNormalization(axis=-1),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    Conv2D(64, [5, 5], activation='relu'),
    BatchNormalization(axis=-1),
    Conv2D(64, [5, 5], activation='relu'),
    BatchNormalization(axis=-1),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

In [9]:
model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [14]:
train_labels_cat.shape

(60000, 10)

In [15]:
model2.fit(train_images_reshape,
          train_labels,
          validation_data=(test_images_reshape, test_labels),
          epochs=10,
          batch_size=25)

Train on 60000 samples, validate on 10000 samples
Epoch 1/25
60000/60000 [==============================] - 131s 2ms/sample - loss: 0.6151 - acc: 0.7867 - val_loss: 0.3894 - val_acc: 0.8568
Epoch 2/25
60000/60000 [==============================] - 154s 3ms/sample - loss: 0.3926 - acc: 0.8591 - val_loss: 0.3590 - val_acc: 0.8630s - loss: 0.3923 - 
Epoch 3/25
60000/60000 [==============================] - 127s 2ms/sample - loss: 0.3447 - acc: 0.8770 - val_loss: 0.2834 - val_acc: 0.8968454
Epoch 4/25
60000/60000 [==============================] - 125s 2ms/sample - loss: 0.3106 - acc: 0.8899 - val_loss: 0.2764 - val_acc: 0.8960 0.88
Epoch 5/25
60000/60000 [==============================] - 126s 2ms/sample - loss: 0.2862 - acc: 0.8984 - val_loss: 0.2655 - val_acc: 0.9031 -
Epoch 6/25
60000/60000 [==============================] - 126s 2ms/sample - loss: 0.2710 - acc: 0.9034 - val_loss: 0.2419 - val_acc: 0.9135
Epoch 7/25
60000/60000 [==============================] - 134s 2ms/sample - loss:

Personal best of
* loss: 0.1303
* acc: 0.9526
* val_loss: 0.2367
* val_acc: 0.9252

with a NN over 25 epochs:
* 2d CNN 32 (5, 5) relu
* normalization, axis=-1
* 2d CNN 32 (5, 5) relu
* normalization, axis=-1
* 2d max pooling (2, 2)
* dropout p = 0.25
* 2d CNN 64 (5, 5) relu
* normalization, axis=-1
* 2d CNN 64 (5, 5) relu
* normalization, axis=-1
* 2d max pooling (2, 2)
* dropout p = 0.25
* flatten
* dense 512 relu
* normalization
* dropout p = 0.5
* dense 10 softmax

This model is clearly overfitting, going to try some regularization

## Model 3 ##

In [17]:
model3 = Sequential([
    BatchNormalization(axis=-1, input_shape=(28, 28, 1)),
    Conv2D(64, [5, 5],
           activation='relu',
          bias_initializer='RandomNormal',
          kernel_initializer='random_uniform'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(512, [5, 5],
           activation='relu'),
    BatchNormalization(axis=-1),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128,
          activation='relu'),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.4),
    Dense(10, activation='softmax')
])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [18]:
model3.compile(loss='sparse_categorical_crossentropy',
               optimizer='adam',
              metrics=['accuracy'])

In [19]:
model3.fit(train_images_reshape, train_labels,
          validation_data=(test_images_reshape, test_labels),
          epochs = 10,
          batch_size = 25)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 398s 7ms/sample - loss: 0.6938 - acc: 0.7628 - val_loss: 0.4642 - val_acc: 0.8376
Epoch 2/10
60000/60000 [==============================] - 385s 6ms/sample - loss: 0.4443 - acc: 0.8483 - val_loss: 0.3419 - val_acc: 0.8782
Epoch 3/10
60000/60000 [==============================] - 410s 7ms/sample - loss: 0.3801 - acc: 0.8697 - val_loss: 0.3189 - val_acc: 0.8848
Epoch 4/10
60000/60000 [==============================] - 422s 7ms/sample - loss: 0.3496 - acc: 0.8798 - val_loss: 0.3261 - val_acc: 0.8859
Epoch 5/10
60000/60000 [==============================] - 408s 7ms/sample - loss: 0.3220 - acc: 0.8882 - val_loss: 0.3368 - val_acc: 0.8878
Epoch 6/10
60000/60000 [==============================] - 396s 7ms/sample - loss: 0.2997 - acc: 0.8957 - val_loss: 0.3090 - val_acc: 0.8891
Epoch 7/10
60000/60000 [==============================] - 400s 7ms/sample - loss: 0.2835 - acc: 0.9020 - val_l